<a href="https://colab.research.google.com/github/Professor-Shin/AI-learning-practice/blob/main/rag_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Step 1: Install Required Libraries
!pip install -q -U google-generativeai langchain-google-genai chromadb pypdf langchain
!pip install -q langchain-community langchain-text-splitters sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 30.2 MB/

In [ ]:
import langchain
print(langchain.__version__)

In [ ]:
# @title Step 2: Setup Google API Key securely
import os
from google.colab import userdata

try:
    # ดึง Key จาก Colab Secrets (วิธีที่แนะนำ)
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("API Key ถูกตั้งค่าเรียบร้อยแล้ว")
except ImportError:
    # กรณีไม่ได้ใช้ Secrets ให้ใส่ Key ตรงๆ (ไม่แนะนำสำหรับงานจริง)
    print("ไม่พบ Colab Secrets, กรุณาใส่ Key โดยตรงใน Code (ถ้าจำเป็น)")
    # os.environ["GOOGLE_API_KEY"] = "PASTE_YOUR_KEY_HERE" # ระวัง Key หลุด
except KeyError:
    print("Error: กรุณาสร้าง Secret ชื่อ 'GOOGLE_API_KEY' ในแถบด้านซ้ายก่อน")

API Key ถูกตั้งค่าเรียบร้อยแล้ว


In [ ]:
# @title Step 3: Load and Split the PDF Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 1. Loading ---
# บอก LangChain ว่าไฟล์เราอยู่ที่ไหน
loader = PyPDFLoader('/content/ABC Company.pdf')
# สั่งให้โหลดเนื้อหาออกมา
documents = loader.load()
print(f"โหลดไฟล์สำเร็จ: มีทั้งหมด {len(documents)} หน้า")


# --- 2. Splitting (Chunking) ---
# เราต้องหั่นข้อความยาวๆ ให้สั้นลง เพื่อให้ AI อ่านรู้เรื่องและประหยัด Token
# chunk_size=500: แต่ละชิ้นมีความยาวประมาณ 500 ตัวอักษร
# chunk_overlap=50: ให้แต่ละชิ้นมีเนื้อหาซ้อนทับกัน 50 ตัวอักษร (เพื่อให้ประโยคไม่ขาดตอน)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# เริ่มทำการหั่น
split_docs = text_splitter.split_documents(documents)

print(f"\n--- ผลลัพธ์การหั่น ---")
print(f"จากเอกสารต้นฉบับ หั่นออกมาได้ทั้งหมด: {len(split_docs)} ชิ้น (Chunks)")

# ลองดูตัวอย่างชิ้นที่ 2 (Index 1) ว่าหน้าตาเป็นยังไง
print(f"\nตัวอย่างเนื้อหาชิ้นที่ 2:\n---")
print(split_docs[1].page_content)
print("---")
print(f"ที่มาของชิ้นนี้: {split_docs[1].metadata}")

โหลดไฟล์สำเร็จ: มีทั้งหมด 1 หน้า

--- ผลลัพธ์การหั่น ---
จากเอกสารต้นฉบับ หั่นออกมาได้ทั้งหมด: 3 ชิ้น (Chunks)

ตัวอย่างเนื้อหาชิ้นที่ 2:
---
4. Equipment and Security Employees must use company-issued laptops for work. Use of 
personal devices for accessing company data is strictly prohibited. Employees must ensure a 
secure internet connection (no public Wi-Fi without VPN).  
5. Communication Employees must check Slack and Email regularly. Urgent matters should 
be communicated via phone call.  
6. Performance Monitoring Managers will review output weekly. Consistent failure to meet
---
ที่มาของชิ้นนี้: {'producer': 'Adobe PDF Library 11.0', 'creator': 'Acrobat PDFMaker 11 for Word', 'creationdate': '2025-12-19T04:07:19+07:00', 'author': 'Siwakorn Saiphaisri', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-12-19T04:07:24+07:00', 'sourcemodified': 'D:20251218210653', 'subject': '', 'title': '', 'source': '/content/ABC Company.pdf', 'total_pages': 1, 'page': 0, 'page_l

In [ ]:
# @title Step 4: Create Embeddings and Store in Vector DB
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 1. นิยามโมเดลที่จะใช้แปลงข้อความเป็นตัวเลข (Embedding Model)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. นำข้อมูลที่หั่นแล้ว (split_docs) ไปใส่ใน ChromaDB
# กระบวนการนี้จะส่งข้อความไปให้ Google แปลงเป็นตัวเลขแล้วเก็บไว้ใน RAM ของ Colab
vector_db = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory="./chroma_db" # เก็บข้อมูลไว้ในโฟลเดอร์นี้
)

print("สร้าง Vector Database และเก็บข้อมูลสำเร็จแล้ว!")

/tmp/ipython-input-4253278828.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


สร้าง Vector Database และเก็บข้อมูลสำเร็จแล้ว!


In [ ]:
# @title Step 5: Setup Chat Interface (RetrievalQA)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.chains.retrieval_qa.base import RetrievalQA


# 1. เรียกใช้ Gemini 2.5 Flash
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# 2. สร้างตัว 'Retriever' (ตัวค้นหาข้อมูล)
# search_kwargs={"k": 2} คือให้ไปหยิบเนื้อหาที่ใกล้เคียงที่สุดมา 2 ชิ้น
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# 3. สร้าง Chain สำหรับการถามตอบ
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # เอา context ทั้งหมด 'ยัด' (stuff) เข้าไปใน Prompt
    retriever=retriever,
    return_source_documents=True # ให้บอกด้วยว่าเอาคำตอบมาจากตรงไหน
)

print("ระบบ Chat Bot พร้อมทำงานแล้ว!")

ระบบ Chat Bot พร้อมทำงานแล้ว!


In [ ]:
# @title Step 6: Test Your RAG System
question = "Who is eligible for WFH and how many days?" # @param {type:"string"}

# รันคำถามผ่านระบบ RAG
result = qa_chain.invoke({"query": question})

print(f"คำถาม: {question}")
print(f"คำตอบจาก Gemini: {result['result']}")
print("\n--- อ้างอิงจากข้อมูลส่วนนี้ ---")
for doc in result['source_documents']:
    print(f"- {doc.page_content}")

คำถาม: Who is eligible for WFH and how many days?
คำตอบจาก Gemini: Full-time employees who have completed their probation period are eligible for up to 2 days of WFH per week, subject to manager approval.

--- อ้างอิงจากข้อมูลส่วนนี้ ---
- ABC Company - Work From Home (WFH) Policy 2024  
1. Purpose This policy outlines the guidelines for employees working remotely to ensure 
productivity and data security.  
2. Eligibility Full-time employees who have completed their probation period are eligible for 
up to 2 days of WFH per week, subject to manager approval.  
3. Working Hours Remote employees must be available during core business hours (10:00 
AM - 4:00 PM). They are expected to log 8 hours of work per day.
- ABC Company - Work From Home (WFH) Policy 2024  
1. Purpose This policy outlines the guidelines for employees working remotely to ensure 
productivity and data security.  
2. Eligibility Full-time employees who have completed their probation period are eligible for 
up to 2 day